In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request,urllib.parse , requests
import re 
import geocoder

- Read the table data from wikipedia using BeautifulSoup and convert it to a dataframe using `pd.read_html` 

- Create a dictionary and loop through all cells and use `regex` to extract the data for the necessary columns since each cell has data in text fomart
- First 3 letters are the **PostalCode**
- **Boroughs** are the next letters before the bracket
- **Neighbourhoods** are inside the brackets
- Replace the slashes with commas and set the Neighbourhoods as one string

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = urllib.request.urlopen(url).read()
soup = BeautifulSoup(data,'lxml')
soup_data = soup.find_all('table')

df = pd.read_html(str(soup_data))[0]

data_dict = {
    'PostalCode':[],
    'Borough':[],
    'Neighborhood':[]
    }

for i in range(len(df.columns)):
    for j in range(len(df)):
        postal_code = df[i][j][0:3]
        borough = re.findall('^[^\(]+',df[i][j][3:] )[0] 
        if borough != 'Not assigned':
            neighborhoods = re.findall('(?<=[(])[^\)]+',df[i][j][3:])[0].split(' / ')
            data_dict['PostalCode'].append(postal_code)
            data_dict['Borough'].append(borough)
            data_dict['Neighborhood'].append(','.join(neighborhoods)) # concat all the list elements into one string

dataframe = pd.DataFrame(data_dict)

We will download the **GeoSpatialDataset.csv** file and concat it with the wikipedia data to get all geodata for all available postal codes

In [3]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'

r = requests.get(url, allow_redirects=True)
open('GeoSpatialDataset.csv', 'wb').write(r.content)

geodata = pd.read_csv('GeoSpatialDataset.csv')
geodata = geodata.rename(columns = {'Postal Code':'PostalCode'})
full_data = pd.merge(dataframe,geodata ,on='PostalCode')
full_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
